In [ ]:
# All imports (including your existing ones)
import json
import requests
import pandas as pd
import chess.pgn
import io
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import chess.engine
import sys
import logging
import math
from enum import Enum
from time import sleep

# Your existing configuration
logging.basicConfig(
    level=logging.INFO, format="%(levelname)s: %(message)s", stream=sys.stdout
)
sns.set(style="whitegrid")
GENERAL_PGN_FILE_PATH = "/Users/benjaminrosales/Desktop/Chess Study Materials & Data/Comparison Games/lichess_db_standard_rated_2017-05.pgn"
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"
ADHD_USERNAMES = ["teoeo", "Tobermorey", "apostatlet", "LovePump1000", "Stuntmanandy",
                  "Banfy_B", "ChessyChesterton12", "Yastoon", "Timy1976", "SonnyDayz11", "xiroir"]

In [ ]:
# Elocator Integration Functions
def get_position_complexity_with_retry(fen, max_retries=3):
    """Get position complexity from Elocator API with retry logic"""
    for attempt in range(max_retries):
        try:
            response = requests.post(
                "https://elocator.fly.dev/complexity/",
                json={"fen": fen}
            )
            if response.status_code == 200:
                sleep(0.1)  # 100ms delay between requests
                return response.json()["complexity_score"]
            logging.warning(f"API returned status code {response.status_code}")
        except Exception as e:
            logging.warning(f"API request failed (attempt {attempt + 1}/{max_retries}): {e}")
        sleep(1)  # Wait longer between retries
    return None

def analyze_game_complexity(pgn):
    """Get full game analysis from Elocator API"""
    try:
        response = requests.post(
            "https://elocator.fly.dev/analyze-game/",
            json={"pgn": pgn}
        )
        return response.json() if response.status_code == 200 else None
    except Exception as e:
        logging.warning(f"Failed to analyze game complexity: {e}")
        return None

In [ ]:
[Your existing utility functions cell - no changes needed]

In [ ]:
def process_games(games, group_label, engine, max_depth=2):
    all_moves = []
    for game in tqdm(games, desc=f"Processing {group_label} games"):
        try:
            # [Your existing game setup code]
            board = game.board()
            game_id = game.headers.get("Site", "Unknown")
            # [Rest of your header extractions]

            node = game
            move_number = 0
            prev_evaluation = None
            prev_time_remaining = None
            prev_winning_chances = None

            # Check for evaluations
            if not any("%eval" in n.comment for n in game.mainline()):
                continue

            while node.variations:
                next_node = node.variations[0]
                move = next_node.move
                
                # Get position complexity before making the move
                current_fen = board.fen()
                complexity_score = get_position_complexity_with_retry(current_fen)
                
                # [Your existing move processing code]
                
                move_data = {
                    # [Your existing move_data fields]
                    'PositionComplexity': position_complexity,  # Your existing complexity measure
                    'ElocatorComplexity': complexity_score,    # New Elocator complexity score
                    # [Rest of your move_data fields]
                }

                all_moves.append(move_data)
                
                # Update board and previous values
                board.push(move)
                node = next_node
                prev_evaluation = evaluation
                prev_time_remaining = time_remaining
                prev_winning_chances = winning_chances

        except Exception as e:
            logging.error(f"Error processing game {game_id}: {e}")
            continue

    moves_df = pd.DataFrame(all_moves)
    return moves_df

# Update column order to include new complexity measure
column_order = [
    'GameID', 'Event', 'Date', 'Result',
    'White', 'Black', 'WhiteElo', 'BlackElo', 'ADHDPlayer',
    'MoveNumber', 'Player', 'SAN', 'GamePhase', 'IsADHDMove',
    'Evaluation', 'EvalChange', 'ErrorCategory',
    'PositionComplexity', 'ElocatorComplexity',  # Added ElocatorComplexity
    'MaterialDiff', 'IsSacrifice',
    'TimeControl', 'TimeControlCategory', 'InitialTimeSeconds',
    'IncrementSeconds', 'TimeRemaining', 'TimeSpent', 'UnderTimePressure',
    'Group', 'MoveCondition'
]

In [ ]:
[Your existing data processing cell - no changes needed]

In [ ]:
# Update statistical analysis functions to include complexity analysis
def analyze_adhd_performance(df):
    """Analyze performance differences between ADHD and control moves"""
    # [Your existing analysis code]
    
    # Add complexity analysis
    print("\nComplexity Analysis:")
    print("-" * 50)
    for group in ['ADHD', 'Control']:
        group_moves = df[df['Group'] == group]
        complexity_stats = group_moves['ElocatorComplexity'].describe()
        print(f"\n{group} Position Complexity:")
        print(f"Mean: {complexity_stats['mean']:.2f}")
        print(f"Median: {complexity_stats['50%']:.2f}")
        print(f"Std Dev: {complexity_stats['std']:.2f}")
        
        # Calculate error rates for different complexity levels
        complexity_bins = pd.qcut(group_moves['ElocatorComplexity'], q=4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
        error_rates = group_moves.groupby(complexity_bins)['ErrorCategory'].apply(
            lambda x: (x != 'Normal').mean()
        )
        print(f"\nError Rates by Complexity Level:")
        for level, rate in error_rates.items():
            print(f"{level} Complexity: {rate:.2%}")

def statistical_comparison(df):
    """Perform statistical tests comparing ADHD and control moves"""
    # [Your existing statistical tests]
    
    # Add complexity comparison
    adhd_complexity = df[df['Group'] == 'ADHD']['ElocatorComplexity'].dropna()
    control_complexity = df[df['Group'] == 'Control']['ElocatorComplexity'].dropna()
    
    # Test for complexity differences
    complexity_stat, complexity_p = stats.mannwhitneyu(
        adhd_complexity,
        control_complexity,
        alternative='two-sided'
    )
    print(f"\nComplexity Analysis:")
    print(f"Mann-Whitney U test for position complexity: p-value = {complexity_p:.4f}")
    
    # Correlation between complexity and errors
    complexity_error_corr = df['ElocatorComplexity'].corr(df['ErrorCategory'].map({'Normal': 0, 'Inaccuracy': 1, 'Mistake': 2, 'Blunder': 3}))
    print(f"Correlation between complexity and error severity: {complexity_error_corr:.4f}")

In [ ]:
# Add complexity visualization functions
def plot_complexity_analysis(df):
    """Create visualizations for complexity analysis"""
    # Distribution of complexity scores
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Group', y='ElocatorComplexity', data=df)
    plt.title('Position Complexity Distribution by Group')
    plt.show()
    
    # Complexity vs Error Rate
    plt.figure(figsize=(12, 6))
    complexity_bins = pd.qcut(df['ElocatorComplexity'], q=10)
    error_rates = df.groupby([complexity_bins, 'Group'])['ErrorCategory'].apply(
        lambda x: (x != 'Normal').mean()
    ).unstack()
    error_rates.plot(kind='line', marker='o')
    plt.title('Error Rates by Position Complexity')
    plt.xlabel('Complexity Level (Percentile)')
    plt.ylabel('Error Rate')
    plt.show()
    
    # Time Spent vs Complexity
    plt.figure(figsize=(12, 6))
    sns.scatterplot(data=df, x='ElocatorComplexity', y='TimeSpent', hue='Group', alpha=0.5)
    plt.title('Time Spent vs Position Complexity')
    plt.show()

    # Complexity by Game Phase
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='GamePhase', y='ElocatorComplexity', hue='Group', data=df)
    plt.title('Position Complexity by Game Phase')
    plt.show()